# Network diagrams for NL/GQL

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import networkx as nwkx

#### Define modes function to get all modes in an nx-ny grid

In [ ]:
def modes(nx,ny):
        
    i = 0
    x = []
    for p in np.arange(-nx,nx+1):
        for q in np.arange(-ny,ny+1):

            x.append((p,q))
            i += 1
    
    return x

In [ ]:
print(modes(1,1))

#### Define triads function to get all feasible triads in an nx-ny grid

In [ ]:
def triads(nx,ny):
        
    x = []
    for p in modes(nx,ny):
        for q in modes(nx,ny):
                        
            k = (p[0]+q[0],p[1]+q[1])
            
            if not(k[0]==0 and k[1]==0) and not(p[0]==0 and p[1]==0) and not(q[0]==0 and q[1]==0): # non-zeros
          
                if np.abs(k[0])<=nx and np.abs(k[1])<=ny: # remove out of bounds
                   
                    if p != q and q != k: # no self-interactions
                    
                        x.append((p,q,k)) # pxq -> k
    
    return x

In [ ]:
print(len(triads(1,1)))

In [ ]:
print(triads(1,1))

In [ ]:
((0,1),(0,-1),(0,0)) in triads(1,1)

In [ ]:
m = modes(1,1)[3]

print("Looking for mode: ", m)

for t in triads(1,1):
    if(m in t):
        print("Present in triad:", t)

#### Edges between modes and triads based on adjacency

In [ ]:
def edges(nx,ny,A):
        
    m = modes(nx,ny)
    
    e = []
    for t in triads(nx,ny):
        
        i = m.index(t[0])
        j = m.index(t[1])
        
        if(A[i,j] > 0.0):
            e.append((t[0],t))
            e.append((t[1],t))
            e.append((t[2],t))
    
    
    return e

In [ ]:
A = np.zeros((9,9))
A[2,3] = 1.0
e = edges(1,1,A)

In [ ]:
print(e)

#### Graph based on edge list between modes and triads

In [ ]:
def graph(nx,ny,A):

    m = modes(nx,ny)
    m.remove((0,0))

    t = triads(nx,ny)
    e = edges(nx,ny,A)

    bpg=nwkx.Graph()

    bpg.add_nodes_from(m,bipartite=0)
    bpg.add_nodes_from(t,bipartite=1)
    bpg.add_edges_from(e)

    dl = list(nwkx.connected_components(bpg))[1:]
    for d in dl:
        for n in d:
            bpg.remove_node(n)

    return bpg

#### Node positions in graph drawing

In [ ]:
def nodepos(G):
    
    l, r = nwkx.bipartite.sets(G)
    pos = {}

    pos.update((node, (0, index)) for index, node in enumerate(l))
    pos.update((node, (1, index)) for index, node in enumerate(r))

    return pos

#### Adjacency matrix labels

In [ ]:
def ax_adj(ax,nx,ny):
    
    l = []
    for m in modes(nx,ny):
        l.append(str(m))

    for a in ax:
        a.set_xticks(np.arange(0,(2*nx+1)*(2*ny+1),1))
        a.set_yticks(np.arange(0,(2*nx+1)*(2*ny+1),1))
        a.set_xticklabels(l,fontsize=14)
        a.set_yticklabels(l,fontsize=14)

### Draw fully connected bipartite network

In [ ]:
A = np.ones((9,9))
G = graph(1,1,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(7,7))

nwkx.draw(G,pos=pos,node_size=25,with_labels=True,width=0.75)

plt.show()

### Draw bipartite network based on  adjacency for 2x2 grid

In [ ]:
dn = "data/"

#### NL

In [ ]:
fn = dn+"adjacency_nl_2x2.npz"
nl = np.load(fn,allow_pickle=True) 

nx = 1
ny = 1

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

im = ax[0].imshow(np.abs(nl['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$|B_{mn}|$',fontsize=15)

im = ax[1].imshow(np.angle(nl['A']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'$\angle B_{mn}$',fontsize=15)

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(nl['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(nl['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = nl['C']

G = graph(1,1,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(7,7))

nwkx.draw(G,pos=pos,node_size=25,with_labels=True,width=0.75)

plt.show()

#### QL

In [ ]:
fn = dn+"adjacency_ql_2x2.npz"
ql = np.load(fn,allow_pickle=True) 

nx = 1
ny = 1

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

im = ax[0].imshow(np.abs(ql['A']),cmap="Reds",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[0])
ax[0].set_title(r'$|B_{mn}|$',fontsize=15)

im = ax[1].imshow(np.angle(ql['A']),cmap="RdBu_r",origin="lower",interpolation="none")
fig.colorbar(im,ax=ax[1])
ax[1].set_title(r'$\angle B_{mn}$',fontsize=15)

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(ql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(ql['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = ql['C']
G = graph(1,1,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(7,7))

nwkx.draw(G,pos=pos,node_size=25,with_labels=True,width=0.75)

plt.show()

### Draw bipartite network based on  adjacency for 3x3 grid

In [ ]:
fn = dn+"adjacency_nl_3x3.npz"
nl = np.load(fn,allow_pickle=True) 

nx = 2
ny = 2

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(nl['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(nl['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = nl['C'] # test a fully connected graph

G = graph(2,2,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(7,7))

nwkx.draw(G,pos=pos,node_size=25,with_labels=False,width=0.75)

plt.show()

In [ ]:
fn = dn+"adjacency_ql_3x3.npz"
ql = np.load(fn,allow_pickle=True) 

nx = 2
ny = 2

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(ql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(ql['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = ql['C'] # test a fully connected graph

G = graph(2,2,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(8,8))

nwkx.draw(G,pos=pos,node_size=25,with_labels=False,width=0.75)

plt.show()

In [ ]:
fn = dn+"adjacency_gql2_3x3.npz"
gql = np.load(fn,allow_pickle=True) 

nx = 2
ny = 2

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(16,6))

# abs
im = ax[0].imshow(np.abs(gql['C']),cmap="nipy_spectral_r",origin="lower",interpolation="none")

ax[0].set_title(r'$|C_{mn}|$',fontsize=15)
fig.colorbar(im,ax=ax[0])

# angle
im = ax[1].imshow(np.angle(gql['C']),cmap="RdBu_r",origin="lower",interpolation="none")

ax[1].set_title(r'$\angle C_{mn}$',fontsize=15)
fig.colorbar(im,ax=ax[1])

ax_adj(ax,nx,ny)

plt.show()

In [ ]:
A = gql['C'] # test a fully connected graph

G = graph(2,2,A)
pos = nodepos(G)

fig,ax = plt.subplots(figsize=(8,8))

nwkx.draw(G,pos=pos,node_size=25,with_labels=False,width=0.75)

plt.show()